In [1]:
# Sema Yalçın - 20120205034
import numpy as np
import pandas as pd
import seaborn as sns
import os
import cv2
import random
import zipfile
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, GlobalAveragePooling2D
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.applications import InceptionV3, ResNet50

In [2]:
zip_file_path = "dataset.zip"
extract_folder = "extracted_images"

# Dosyayı zipten çıkardım.
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # SolvaDataset_200_v3 kalörünü çıkardım.
    zip_ref.extractall(extract_folder)

In [3]:
# Görüntülerin dosya yollarını sakladım.
image_paths = []
root_folder = os.path.join(extract_folder, "SolvaDataset_200_v3")

for root, dirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith(".bmp"):
            image_path = os.path.join(root, file)
            image_paths.append(image_path)

random.shuffle(image_paths)

In [4]:
# Train ve test olarak ayırdım.
train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)

train_images = []
train_labels = []
test_images = []
test_labels = []

# Train ve test verilerini sınıflara göre ayırdım ve class bilgilerini oluşturdum.
for path in train_paths:
    folder_name = os.path.basename(os.path.dirname(path))
    image = Image.open(path).convert('L')
    train_images.append(np.array(image))
    train_labels.append(folder_name)

for path in test_paths:
    folder_name = os.path.basename(os.path.dirname(path))
    image = Image.open(path).convert('L')
    test_images.append(np.array(image))
    test_labels.append(folder_name)

# Class bilgilerini one-hot-encoding yaptım.
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

onehot_encoder = OneHotEncoder(sparse=False)
train_labels_encoded = onehot_encoder.fit_transform(train_labels_encoded.reshape(-1, 1))
test_labels_encoded = onehot_encoder.transform(test_labels_encoded.reshape(-1, 1))

In [5]:
# Train ve test verilerini numpy dizisine aldım.
x_train = np.array(train_images)
y_train = np.array(train_labels_encoded)
x_test = np.array(test_images)
y_test = np.array(test_labels_encoded)

# Train ve test verilerini reshape yaptım.
x_train = x_train.reshape(-1, 120, 120, 1)
x_test = x_test.reshape(-1, 120, 120, 1)

In [6]:
x_train.shape

(2361, 120, 120, 1)

In [7]:
x_test.shape

(591, 120, 120, 1)

In [8]:
y_train.shape

(2361, 15)

In [9]:
y_test.shape

(591, 15)

In [10]:
# LeNet mimarisi
model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), padding = 'same', activation="relu", input_shape=(120, 120, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(5, 5), padding = 'valid', activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation="relu"))
model.add(Dense(84, activation="relu"))
model.add(Dense(15, activation="softmax"))

model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 120, 6)       156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 60, 60, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 120)               1

In [11]:
# Model derleme
adam = Adam(learning_rate = 5e-4)
model.compile(loss="categorical_crossentropy", optimizer = adam, metrics=["accuracy"])

In [12]:
# Learning rate azaltmak için geri çağırma uyguladım.
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=3, verbose=1, min_lr = 1e-6)

In [13]:
x_train.shape

(2361, 120, 120, 1)

In [14]:
y_train.shape

(2361, 15)

In [16]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), callbacks=[reduce_lr])

Epoch 1/10
74/74 [==============================] - 9s 114ms/step - loss: 5.5535 - accuracy: 0.2973 - val_loss: 1.9120 - val_accuracy: 0.3976 - lr: 5.0000e-04
Epoch 2/10
74/74 [==============================] - 8s 114ms/step - loss: 1.1987 - accuracy: 0.6184 - val_loss: 1.5635 - val_accuracy: 0.5347 - lr: 5.0000e-04
Epoch 3/10
74/74 [==============================] - 12s 163ms/step - loss: 0.5396 - accuracy: 0.8386 - val_loss: 1.5399 - val_accuracy: 0.5719 - lr: 5.0000e-04
Epoch 4/10
74/74 [==============================] - 13s 170ms/step - loss: 0.1991 - accuracy: 0.9504 - val_loss: 1.5515 - val_accuracy: 0.6142 - lr: 5.0000e-04
Epoch 5/10
74/74 [==============================] - 14s 184ms/step - loss: 0.0711 - accuracy: 0.9911 - val_loss: 1.6877 - val_accuracy: 0.5990 - lr: 5.0000e-04
Epoch 6/10
74/74 [==============================] - 14s 188ms/step - loss: 0.0265 - accuracy: 0.9983 - val_loss: 1.7206 - val_accuracy: 0.6091 - lr: 5.0000e-04
Epoch 7/10
74/74 [========================

In [18]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)

print("Test Loss:", test_loss)
print("Test Accuracy: {:.2%}".format(test_accuracy))

19/19 - 0s - loss: 1.9665 - accuracy: 0.6261 - 472ms/epoch - 25ms/step
Test Loss: 1.9665337800979614
Test Accuracy: 62.61%


In [19]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

y_true = np.argmax(y_test, axis=1)

# Kesinlik
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)

# Geri çağırma
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

# F-ölçüsü
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

print("Kesinlik: ", precision)
print("Geri Çağırma: ", recall)
print("F-ölçüsü: ", f1)

19/19 [==============================] - 1s 25ms/step
Kesinlik:  0.6391738774874531
Geri Çağırma:  0.626057529610829
F-ölçüsü:  0.626782703497515


In [15]:
# Google-net modeli için ayarlamalr yaptım. Görüntüyü rgb görüntüsüne dönüştürdüm.
input_shape = (120, 120, 3)

x_train_rgb = [cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) for img in x_train]
x_test_rgb = [cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) for img in x_test]

base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

In [16]:
# Google-net modeli
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(15, activation='softmax')(x)

model_google_net = Model(inputs=base_model.input, outputs=predictions)

In [17]:
x_train_rgb = np.array(x_train_rgb)
x_test_rgb = np.array(x_test_rgb)
x_test_rgb.shape

(591, 120, 120, 3)

In [23]:
# Modeli derle
model_google_net.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli fit et
model_google_net.fit(x_train_rgb, y_train, epochs=10, batch_size=32, validation_data=(x_test_rgb, y_test))

Epoch 1/10
74/74 [==============================] - 44s 538ms/step - loss: 46.7743 - accuracy: 0.3727 - val_loss: 5.6285 - val_accuracy: 0.5398
Epoch 2/10
74/74 [==============================] - 42s 576ms/step - loss: 3.8775 - accuracy: 0.6019 - val_loss: 3.1562 - val_accuracy: 0.5922
Epoch 3/10
74/74 [==============================] - 40s 545ms/step - loss: 2.3511 - accuracy: 0.6222 - val_loss: 2.7406 - val_accuracy: 0.5770
Epoch 4/10
74/74 [==============================] - 41s 562ms/step - loss: 1.6095 - accuracy: 0.6650 - val_loss: 2.1729 - val_accuracy: 0.5685
Epoch 5/10
74/74 [==============================] - 43s 580ms/step - loss: 1.0151 - accuracy: 0.7039 - val_loss: 1.6535 - val_accuracy: 0.5956
Epoch 6/10
74/74 [==============================] - 44s 595ms/step - loss: 0.7950 - accuracy: 0.7590 - val_loss: 1.3095 - val_accuracy: 0.6464
Epoch 7/10
74/74 [==============================] - 38s 509ms/step - loss: 0.6588 - accuracy: 0.7798 - val_loss: 1.3450 - val_accuracy: 0.661

In [30]:
test_loss, test_accuracy = model_google_net.evaluate(x_test_rgb, y_test, verbose=2)

print("Test Loss:", test_loss)
print("Test Accuracy: {:.2%}".format(test_accuracy))

19/19 - 5s - loss: 1.3327 - accuracy: 0.6582 - 5s/epoch - 263ms/step
Test Loss: 1.3327076435089111
Test Accuracy: 65.82%


In [25]:
y_pred = model_google_net.predict(x_test_rgb)
y_pred = np.argmax(y_pred, axis=1)

# Gerçek sınıf etiketleri
y_true = np.argmax(y_test, axis=1)

# Kesinlik
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)

# Geri çağırma
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

# F-ölçüsü
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

print("Kesinlik: ", precision)
print("Geri Çağırma: ", recall)
print("F-ölçüsü: ", f1)

19/19 [==============================] - 5s 211ms/step
Kesinlik:  0.7000709186972678
Geri Çağırma:  0.6582064297800339
F-ölçüsü:  0.6591980345827791


In [18]:
# Res-net modeli
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(120, 120, 3))

for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(15, activation='softmax')(x)

model_resnet = Model(inputs=base_model.input, outputs=predictions)

In [28]:
model_resnet.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model_resnet.fit(x_train_rgb, y_train, epochs=10, batch_size=32, validation_data=(x_test_rgb, y_test))

Epoch 1/10
74/74 [==============================] - 78s 1s/step - loss: 1.0515 - accuracy: 0.6785 - val_loss: 0.4711 - val_accuracy: 0.8376
Epoch 2/10
74/74 [==============================] - 92s 1s/step - loss: 0.4278 - accuracy: 0.8446 - val_loss: 0.5406 - val_accuracy: 0.8020
Epoch 3/10
74/74 [==============================] - 94s 1s/step - loss: 0.3160 - accuracy: 0.8882 - val_loss: 0.4573 - val_accuracy: 0.8257
Epoch 4/10
74/74 [==============================] - 99s 1s/step - loss: 0.2337 - accuracy: 0.9132 - val_loss: 0.4661 - val_accuracy: 0.8342
Epoch 5/10
74/74 [==============================] - 94s 1s/step - loss: 0.2083 - accuracy: 0.9267 - val_loss: 0.4222 - val_accuracy: 0.8359
Epoch 6/10
74/74 [==============================] - 78s 1s/step - loss: 0.1464 - accuracy: 0.9492 - val_loss: 0.4354 - val_accuracy: 0.8511
Epoch 7/10
74/74 [==============================] - 106s 1s/step - loss: 0.1719 - accuracy: 0.9432 - val_loss: 0.6012 - val_accuracy: 0.8156
Epoch 8/10
74/74 [=

In [31]:
test_loss, test_accuracy = model_resnet.evaluate(x_test_rgb, y_test, verbose=2)

print("Test Loss:", test_loss)
print("Test Accuracy: {:.2%}".format(test_accuracy))

19/19 - 13s - loss: 0.4785 - accuracy: 0.8545 - 13s/epoch - 693ms/step
Test Loss: 0.47846296429634094
Test Accuracy: 85.45%


In [32]:
y_pred = model_resnet.predict(x_test_rgb)
y_pred = np.argmax(y_pred, axis=1)

y_true = np.argmax(y_test, axis=1)

# Kesinlik
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)

# Geri çağırma
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

# F-ölçüsü
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

print("Kesinlik: ", precision)
print("Geri Çağırma: ", recall)
print("F-ölçüsü: ", f1)

19/19 [==============================] - 15s 704ms/step
Kesinlik:  0.8820852634938682
Geri Çağırma:  0.8544839255499154
F-ölçüsü:  0.85562753414163


In [19]:
# 1. modelim
model_special = Sequential()
model_special.add(Conv2D(10, kernel_size=(5, 5), padding = 'same', activation="sigmoid", input_shape=(120, 120, 1)))
model_special.add(MaxPooling2D(pool_size=(3, 3)))
model_special.add(Conv2D(20, kernel_size=(5, 5), padding = 'valid', activation="sigmoid"))
model_special.add(MaxPooling2D(pool_size=(3, 3)))
model_special.add(Flatten())
model_special.add(Dense(120, activation="relu"))
model_special.add(Dense(84, activation="relu"))
model_special.add(Dense(15, activation="softmax"))

model_special.build()
model_special.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_96 (Conv2D)          (None, 120, 120, 10)      260       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 40, 40, 10)       0         
 2D)                                                             
                                                                 
 conv2d_97 (Conv2D)          (None, 36, 36, 20)        5020      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 20)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2880)              0         
                                                                 
 dense_7 (Dense)             (None, 120)              

In [20]:
model_special.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_special.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), callbacks=[reduce_lr])

Epoch 1/10
74/74 [==============================] - 8s 99ms/step - loss: 2.7440 - accuracy: 0.0640 - val_loss: 2.7141 - val_accuracy: 0.0474 - lr: 0.0010
Epoch 2/10
74/74 [==============================] - 9s 128ms/step - loss: 2.7000 - accuracy: 0.0873 - val_loss: 2.6907 - val_accuracy: 0.0778 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - 11s 151ms/step - loss: 2.6204 - accuracy: 0.1482 - val_loss: 2.5675 - val_accuracy: 0.2267 - lr: 0.0010
Epoch 4/10
74/74 [==============================] - 11s 143ms/step - loss: 2.2931 - accuracy: 0.2690 - val_loss: 2.1224 - val_accuracy: 0.3096 - lr: 0.0010
Epoch 5/10
74/74 [==============================] - 10s 134ms/step - loss: 1.8772 - accuracy: 0.3761 - val_loss: 1.7456 - val_accuracy: 0.4095 - lr: 0.0010
Epoch 6/10
74/74 [==============================] - 9s 128ms/step - loss: 1.6411 - accuracy: 0.4485 - val_loss: 1.6337 - val_accuracy: 0.4585 - lr: 0.0010
Epoch 7/10
74/74 [==============================] - 9s 128ms/step - 

In [21]:
test_loss, test_accuracy = model_special.evaluate(x_test, y_test, verbose=2)

print("Test Loss:", test_loss) 
print("Test Accuracy: {:.2%}".format(test_accuracy))

19/19 - 0s - loss: 1.1928 - accuracy: 0.6159 - 452ms/epoch - 24ms/step
Test Loss: 1.19280207157135
Test Accuracy: 61.59%


In [23]:
y_pred = model_special.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

y_true = np.argmax(y_test, axis=1)

# Kesinlik
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)

# Geri çağırma
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

# F-ölçüsü
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

print("Kesinlik: ", precision)
print("Geri Çağırma: ", recall)
print("F-ölçüsü: ", f1)

19/19 [==============================] - 1s 23ms/step
Kesinlik:  0.6551308768710135
Geri Çağırma:  0.6159052453468697
F-ölçüsü:  0.6128842864019378


In [24]:
# 2. modelim
model_special2 = Sequential()
model_special2.add(Conv2D(10, kernel_size=(5, 5), padding = 'same', activation="relu", input_shape=(120, 120, 1)))
model_special2.add(MaxPooling2D(pool_size=(3, 3)))
model_special2.add(Conv2D(20, kernel_size=(5, 5), padding = 'valid', activation="relu"))
model_special2.add(MaxPooling2D(pool_size=(3, 3)))
model_special2.add(Conv2D(40, kernel_size=(5, 5), padding = 'valid', activation="relu"))
model_special2.add(MaxPooling2D(pool_size=(3, 3)))
model_special2.add(Flatten())
model_special2.add(Dense(120, activation="relu"))
model_special2.add(Dense(84, activation="relu"))
model_special2.add(Dense(15, activation="softmax"))

model_special2.build()
model_special2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_98 (Conv2D)          (None, 120, 120, 10)      260       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 40, 40, 10)       0         
 2D)                                                             
                                                                 
 conv2d_99 (Conv2D)          (None, 36, 36, 20)        5020      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 12, 12, 20)       0         
 2D)                                                             
                                                                 
 conv2d_100 (Conv2D)         (None, 8, 8, 40)          20040     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 2, 2, 40)        

In [25]:
model_special2.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_special2.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), callbacks=[reduce_lr])

Epoch 1/10
74/74 [==============================] - 8s 99ms/step - loss: 4.9418 - accuracy: 0.1698 - val_loss: 2.1271 - val_accuracy: 0.3215 - lr: 0.0010
Epoch 2/10
74/74 [==============================] - 7s 95ms/step - loss: 1.6725 - accuracy: 0.4507 - val_loss: 1.5106 - val_accuracy: 0.4907 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - 9s 123ms/step - loss: 1.1121 - accuracy: 0.6201 - val_loss: 1.1982 - val_accuracy: 0.5990 - lr: 0.0010
Epoch 4/10
74/74 [==============================] - 9s 124ms/step - loss: 0.8180 - accuracy: 0.7243 - val_loss: 1.2005 - val_accuracy: 0.6142 - lr: 0.0010
Epoch 5/10
74/74 [==============================] - 9s 126ms/step - loss: 0.6458 - accuracy: 0.7789 - val_loss: 1.0536 - val_accuracy: 0.6599 - lr: 0.0010
Epoch 6/10
74/74 [==============================] - 9s 124ms/step - loss: 0.5127 - accuracy: 0.8230 - val_loss: 0.8784 - val_accuracy: 0.7107 - lr: 0.0010
Epoch 7/10
74/74 [==============================] - 9s 126ms/step - loss

In [26]:
test_loss, test_accuracy = model_special2.evaluate(x_test, y_test, verbose=2)

print("Test Loss:", test_loss)
print("Test Accuracy: {:.2%}".format(test_accuracy))

19/19 - 0s - loss: 0.8923 - accuracy: 0.7496 - 453ms/epoch - 24ms/step
Test Loss: 0.892257034778595
Test Accuracy: 74.96%


In [27]:
y_pred = model_special2.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

y_true = np.argmax(y_test, axis=1)

# Kesinlik
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)

# Geri çağırma
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

# F-ölçüsü
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

print("Kesinlik: ", precision)
print("Geri Çağırma: ", recall)
print("F-ölçüsü: ", f1)


19/19 [==============================] - 1s 24ms/step
Kesinlik:  0.7619531926825325
Geri Çağırma:  0.7495769881556683
F-ölçüsü:  0.7488824986374076
